In [1]:
import os
import numpy as np
import sklearn
import tensorflow
from keras import Sequential
from keras.layers import Conv1D, Embedding, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import categorical_accuracy
from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn import datasets
import pickle
# import wandb
# from wandb.keras import WandbCallback
# wandb.init(project="nlp")

physical_devices = tensorflow.config.experimental.list_physical_devices('GPU')
print("physical_devices-------------", len(physical_devices))
tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)

datapath = "kategorie_cleaned/"
if "DATA_PATH" in os.environ:
    datapath = os.environ['DATA_PATH']


train = sklearn.datasets.load_files(datapath + "/", description=None, categories=None,
                                    load_content=True,
                                    shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)

texts = train.data  # Extract text
target = train.target  # Extract target


physical_devices------------- 1


In [3]:
from ttictoc import tic, toc
tic()
vocab_size = 996175
tokenizer = Tokenizer(num_words = vocab_size)  # Setup tokenizer
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # Generate sequences
# texts.clear()
word_index = tokenizer.word_index
print('Found {:,} unique words.'.format(len(word_index)))
print(toc())
# word_index.clear()

Found 996,175 unique words.
353.1837554000001


In [4]:
# inv_index = {v: k for k, v in tokenizer.word_index.items()}
avg = sum(map(len, sequences)) / len(sequences)
# Get the standard deviation of the sequence length
std = np.sqrt(sum(map(lambda x: (len(x) - avg) ** 2, sequences)) / len(sequences))
print(avg, std)
max_length = 3500
data = pad_sequences(sequences, maxlen=max_length)
# sequences.clear()
# print(data[0])
# print(texts[0])
# print(sequences)
from tensorflow.keras.utils import to_categorical
labels = to_categorical(np.asarray(target))
print('Shape of data:', data.shape)
print('Shape of labels:', labels.shape)

945.7917857830312 1100.1441532992292
Shape of data: (140269, 3500)
Shape of labels: (140269, 120)


In [5]:
glove_dir = 'C:/Users/Adi/Documents/glove'  # This is the folder with the dataset

embeddings_index = {}  # We create a dictionary of word -> embedding

with open(os.path.join(glove_dir, 'glove_100d_pl.txt'), encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # The first value is the word, the rest are the values of the embedding
        embedding = np.asarray(values[1:], dtype='float16')  # Load embedding
        embeddings_index[word] = embedding  # Add embedding to our embedding dictionary

embedding_dim = 100  # We use 100 dimensional glove vectors

word_index = tokenizer.word_index
nb_words = min(vocab_size, len(word_index))  # How many words are there actually

embedding_matrix = np.zeros((nb_words, embedding_dim))

# The vectors need to be in the same position as their index.
# Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on

# Loop over all words in the word index
for word, i in word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= vocab_size:
        continue
    # Get the embedding vector for the word
    embedding_vector = embeddings_index.get(word)
    # If there is an embedding vector, put it in the embedding matrix
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
word_index.clear()
# print(embedding_matrix[100])

In [6]:
from tensorflow.keras.layers import SpatialDropout1D, LSTM, GlobalMaxPooling1D, BatchNormalization, GRU, Bidirectional
from tensorflow.keras.layers import Conv1D, Embedding, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras import Sequential
from keras.callbacks import EarlyStopping

from keras.backend import sigmoid
def mish(x, beta = 1):
    return (x * tensorflow.math.tanh(tensorflow.keras.activations.softplus(x)))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'mish': Activation(mish)})

# model = Sequential()
# model.add(Embedding(vocab_size,
#                     embedding_dim,
#                     input_length=max_length,
#                     weights=[embedding_matrix],
#                     trainable=False)
# #                     output_dim=120)
#          )
# model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
# model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Conv1D(filters=32, kernel_size=6, padding='same', activation='relu'))
# model.add(MaxPooling1D(3))
# model.add(Dropout(0.3))
# model.add(Bidirectional(GRU(256, dropout=0.3, recurrent_activation="sigmoid", return_sequences=True)))
# model.add(Bidirectional(LSTM(256, dropout=0.3, recurrent_activation="sigmoid")))
# model.add(Dense(128, activation='mish'))
# model.add(GRU(256, dropout=0.5))
# model.add(Dense(120, activation='softmax'))
# model.summary()
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#
# epochs = 15
# batch_size = 64
# history = model.fit(data, labels, epochs=epochs, batch_size=batch_size,validation_split=0.3)

from keras.backend import sigmoid
def mish(x, beta = 1):
    return (x * tensorflow.math.tanh(tensorflow.keras.activations.softplus(x)))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'mish': Activation(mish)})


import matplotlib.pyplot as plt
model = Sequential()
model.add(Embedding(vocab_size,
                    embedding_dim,
                    input_length=max_length,
                    weights=[embedding_matrix],
                    trainable=False
#                     output_dim=125)
         ))

model.add(Conv1D(256, 5, activation='mish'))
model.add(MaxPooling1D(3))
model.add(BatchNormalization())
model.add(Conv1D(128, 5, activation='mish'))
model.add(MaxPooling1D(3))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='mish'))
model.add(BatchNormalization())
model.add(Dense(120, activation='softmax'))

# model.add(Conv1D(128, 12, activation='mish'))
# model.add(Conv1D(64, 12, activation='mish'))
# model.add(MaxPooling1D(3))
# model.add(Flatten())
# model.add(Dropout(0.3))
# model.add(Dense(256, activation='mish'))
# model.add(Dense(125, activation='softmax'))


model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[categorical_accuracy])

history = model.fit(data, labels, validation_split=0.3, epochs=30, batch_size=64
              # ,callbacks=[WandbCallback()]
                   )

# plt.plot(history.history['categorical_accuracy'])
# plt.plot(history.history['val_categorical_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model accuracy')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3500, 100)         99617500  
_________________________________________________________________
conv1d (Conv1D)              (None, 3496, 256)         128256    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1165, 256)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1165, 256)         1024      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1161, 128)         163968    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 387, 128)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 387, 128)          5

KeyboardInterrupt: 

In [7]:
model.save("model_pl.h5")